In [1]:
import pandas as pd
import numpy as np
import random
from IPython.display import display, Markdown

# Tentukan nama file dataset
file_name = "disparbud-od_15367_jml_pengunjung_ke_objek_wisata__jenis_wisatawan_ka_v2_data.csv"

# --- 1. Muat dan Bersihkan Data ---
df = pd.read_csv(file_name)

# Filter data untuk area yang mengandung kata 'BANDUNG' (case-insensitive)
# Ini mencakup KABUPATEN BANDUNG, KOTA BANDUNG, KABUPATEN BANDUNG BARAT, dll.
bandung_df = df[df['nama_kabupaten_kota'].str.contains('BANDUNG', case=False, na=False)].copy()

# Agregasi total pengunjung (domestik & mancanegara) per tahun
yearly_visitors = bandung_df.groupby('tahun')['jumlah_pengunjung'].sum().reset_index()
yearly_visitors.columns = ['Tahun', 'Jumlah Pengunjung']

# Simpan data yang telah diagregasi
yearly_visitors.to_csv("bandung_yearly_visitors_aggregated.csv", index=False)

display(Markdown("### 1. Data Historis Agregasi Pengunjung ke Bandung"))
display(yearly_visitors)

### 1. Data Historis Agregasi Pengunjung ke Bandung

,Tahun,Jumlah Pengunjung
0,2014,8663243
1,2015,8663243
2,2016,9881713
3,2017,7714193
4,2018,18033206
5,2019,10372669
6,2020,7169276
7,2021,4432044
8,2022,10659608
9,2023,7433715


In [2]:
# --- 2. Perhitungan Probabilitas dan Interval ---
visitors_df = yearly_visitors.copy()

# A. Hitung Total Pengunjung Historis
total_visitors = visitors_df['Jumlah Pengunjung'].sum()

# B. Hitung Probabilitas
visitors_df['Probabilitas'] = visitors_df['Jumlah Pengunjung'] / total_visitors

# C. Hitung Kumulatif
visitors_df['Kumulatif'] = visitors_df['Probabilitas'].cumsum()

# D. Hitung Interval (dikonversi ke skala 0-999)
# Batas Bawah: Kumulatif sebelumnya * 1000
visitors_df['Batas Bawah'] = (visitors_df['Kumulatif'].shift(1, fill_value=0) * 1000).round().astype(int)

# Batas Atas: Kumulatif saat ini * 1000, dikurangi 1
visitors_df['Batas Atas'] = (visitors_df['Kumulatif'] * 1000).round().astype(int) - 1

# Pastikan batas bawah pertama adalah 0 dan batas atas terakhir adalah 999
visitors_df.loc[visitors_df.index[0], 'Batas Bawah'] = 0
visitors_df.loc[visitors_df.index[-1], 'Batas Atas'] = 999

# Kolom Interval dalam format string (misal: "0 - 80")
visitors_df['Interval'] = visitors_df.apply(lambda row: f"{row['Batas Bawah']} - {row['Batas Atas']}", axis=1)

# Kolom yang akan ditampilkan untuk Tabel Interval
interval_table = visitors_df[['Tahun', 'Jumlah Pengunjung', 'Probabilitas', 'Kumulatif', 'Interval']]
interval_table['Probabilitas'] = interval_table['Probabilitas'].map('{:.4f}'.format)
interval_table['Kumulatif'] = interval_table['Kumulatif'].map('{:.4f}'.format)

display(Markdown("### 2. Tabel Interval Probabilitas Monte Carlo"))
display(interval_table)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_11476\3381069758.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interval_table['Probabilitas'] = interval_table['Probabilitas'].map('{:.4f}'.format)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_11476\3381069758.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interval_table['Kumulatif'] = interval_table['Kumulatif'].map('{:.4f}'.format)


### 2. Tabel Interval Probabilitas Monte Carlo

,Tahun,Jumlah Pengunjung,Probabilitas,Kumulatif,Interval
0,2014,8663243,0.0808,0.0808,0.0 - 80.0
1,2015,8663243,0.0808,0.1617,81.0 - 161.0
2,2016,9881713,0.0922,0.2539,162.0 - 253.0
3,2017,7714193,0.0720,0.3258,254.0 - 325.0
4,2018,18033206,0.1683,0.4941,326.0 - 493.0
5,2019,10372669,0.0968,0.5909,494.0 - 590.0
6,2020,7169276,0.0669,0.6578,591.0 - 657.0
7,2021,4432044,0.0414,0.6991,658.0 - 698.0
8,2022,10659608,0.0995,0.7986,699.0 - 798.0
9,2023,7433715,0.0694,0.8680,799.0 - 867.0


In [3]:
# --- 3. Fungsi Prediksi dan Simulasi ---

# Fungsi untuk memetakan Angka Acak ke Prediksi Jumlah Pengunjung
def get_prediction(rand_num, df):
    """
    Memetakan bilangan acak (0-999) ke Jumlah Pengunjung berdasarkan interval.
    """
    # Cari baris di mana rand_num berada dalam Batas Bawah dan Batas Atas
    row = df[(df['Batas Bawah'] <= rand_num) & (df['Batas Atas'] >= rand_num)]
    if not row.empty:
        return row['Jumlah Pengunjung'].iloc[0]
    return np.nan

# Tentukan parameter simulasi
NUM_SIMULATIONS = 5
random.seed(42) # Set seed agar hasil dapat direproduksi
random_numbers = [random.randint(0, 999) for _ in range(NUM_SIMULATIONS)]

# Jalankan simulasi
simulation_results = pd.DataFrame({
    'No.': range(1, NUM_SIMULATIONS + 1),
    'Angka Acak (0-999)': random_numbers,
    'Prediksi Jumlah Wisatawan': [get_prediction(rn, visitors_df) for rn in random_numbers]
})

# Hitung rata-rata hasil simulasi sebagai prediksi akhir
final_prediction_value = simulation_results['Prediksi Jumlah Wisatawan'].mean()

display(Markdown("### 3. Hasil Simulasi Monte Carlo"))
display(simulation_results)

display(Markdown(f"""
---
#### Prediksi Akhir
**Jumlah Prediksi Wisatawan ke Bandung di tahun berikutnya (Rata-rata dari {NUM_SIMULATIONS} simulasi) adalah:**
# **{int(round(final_prediction_value)):,} Orang**
*(Jika dibulatkan ke bilangan bulat terdekat)*
---
"""))

### 3. Hasil Simulasi Monte Carlo

,No.,Angka Acak (0-999),Prediksi Jumlah Wisatawan
0,1,654,7169276
1,2,114,8663243
2,3,25,8663243
3,4,759,10659608
4,5,281,7714193



---
#### Prediksi Akhir
**Jumlah Prediksi Wisatawan ke Bandung di tahun berikutnya (Rata-rata dari 5 simulasi) adalah:**
# **8,573,913 Orang**
*(Jika dibulatkan ke bilangan bulat terdekat)*
---
